In [9]:
!apt-get install tesseract-ocr poppler-utils
!pip install nltk scikit-learn pdf2image pytesseract pillow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 1 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 1s (272 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...


In [12]:
import nltk
import re
import pytesseract
import numpy as np

from pdf2image import convert_from_path
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
from google.colab import files
uploaded = files.upload()


Saving EPaper-International_19-12-2025_GQNFASST3.pdf to EPaper-International_19-12-2025_GQNFASST3.pdf


In [10]:
pdf_path = list(uploaded.keys())[0]

images = convert_from_path(pdf_path)

text = ""
for img in images:
    text += pytesseract.image_to_string(img)

text[:1000]


'202 Indians recruited into Russian forces since 2022: MEA\n\nie\n\nVijaita Singh\nNEW DELHI\n\nAs many as 202 Indians are\n“believed to have been re-\ncruited into the Russian\narmed forces” since 2022,\nand 26 of them are report-\ned to have lost their lives,\nthe Ministry of External Af-\nfairs (MEA) informed the\nRajya Sabha on Thursday,\nadding that at least two In-\ndians have been cremated\nin Russia.\n\n“Concerted efforts by\nthe Government have re-\nsulted in early discharge of\n119 of them; 26 are report-\ned to have lost their lives\nand seven are reported\nmissing by the Russian\nside. Efforts are ongoing\nfor early discharge of 50 in-\ndividuals,” Minister of\nState Kirti Vardhan Singh\nsaid in a written reply.\n\nMr. Singh was respond-\ning to a question by Trina-\nmool Congress MP Saket\nGokhale and Congress MP\nRandeep Singh Surjewala\non the number of Indian\nnationals identified as ille-\ngally working or forced to\nwork in the Russian army,\nalong with the count of f

In [13]:
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = re.sub(r'[^a-zA-Z ]', '', text)
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop_words]
    return " ".join(tokens)

cleaned_text = clean_text(text)
cleaned_text[:500]


'indians recruited russian forces since meaievijaita singhnew delhias many indians arebelieved recruited russianarmed forces since reported lost livesthe ministry external affairs mea informed therajya sabha thursdayadding least two indians crematedin russiaconcerted efforts bythe government resulted early discharge reported lost livesand seven reportedmissing russianside efforts ongoingfor early discharge individuals minister ofstate kirti vardhan singhsaid written replymr singh responding quest'

In [14]:
vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform([cleaned_text])


In [15]:
lda = LatentDirichletAllocation(
    n_components=4,
    random_state=42
)

lda.fit(X)


LatentDirichletAllocation(n_components=4, random_state=42)

In [16]:
feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f"\nTopic {topic_idx+1}:")
    print([feature_names[i] for i in topic.argsort()[-10:]])



Topic 1:
['withthe', 'zone', 'return', 'remains', 'russia', 'army', 'minister', 'early', 'enlisted', 'family']

Topic 2:
['withthe', 'zone', 'return', 'remains', 'russia', 'army', 'minister', 'early', 'enlisted', 'family']

Topic 3:
['family', 'early', 'deceased', 'reported', 'nationals', 'mortal', 'two', 'indian', 'indians', 'russian']

Topic 4:
['withthe', 'zone', 'return', 'remains', 'russia', 'army', 'minister', 'early', 'enlisted', 'family']
